### MAST30034: Applied Data Science Project 2
---
# Naive Ranking: Unweighted Average Rank

Calculate the unweights average rank over all merchants (without segmentation).

In [1]:
# useful imports
import pandas as pd
import numpy as np
import os
import re

os.chdir("../scripts")
import utilities.rank_utilities as RANK

In [2]:
# read the curated data
merchant_df = pd.read_parquet('../data/curated/final_merchant_statistics')

In [3]:
merchant_df.columns

Index(['merchant_abn', 'name', 'tags', 'tag', 'revenue_level', 'take_rate',
       'sales_revenue', 'discounted_sales_revenue', 'num_orders',
       'approximate_fraudulent_orders', 'avg_daily_rev',
       'discounted_avg_daily_rev', 'avg_value_per_order',
       'discounted_avg_value_per_order', 'avg_daily_orders',
       'avg_daily_approximate_fraudulent_orders', 'std_daily_revenue',
       'std_daily_discounted_revenue', 'avg_daily_commission',
       'discounted_avg_daily_commission', 'avg_commission_per_order',
       'discounted_avg_commission_per_order', 'overall_commission',
       'discounted_overall_commission', 'overall_fraud_rate',
       'sa2_region_count', 'median_customer_income', 'returning_customers',
       'unique_customers', 'mean_spending', 'std_spending', 'vip_customers'],
      dtype='object')

In [4]:
merchant_df.head(5)

,merchant_abn,name,tags,tag,revenue_level,take_rate,sales_revenue,discounted_sales_revenue,num_orders,approximate_fraudulent_orders,...,overall_commission,discounted_overall_commission,overall_fraud_rate,sa2_region_count,median_customer_income,returning_customers,unique_customers,mean_spending,std_spending,vip_customers
0,10023283211,Felis Limited,"((furniture, home furnishings and equipment sh...","furniture, home furnishings and equipment shop...",e,0.18,7.032777e+05,6.554602e+05,3261.0,216.117177,...,1265.899881,1179.828379,0.066273,1717.0,794.816023,11.0,3032.0,231.951752,155.424439,137.0
1,10346855916,Odio Institute,"((equipment, tool, furniture, and appliance r...","equipment, tool, furniture, and appliance ren...",b,3.57,6.775178e+03,6.166115e+03,6.0,0.492377,...,241.873872,220.130319,0.082063,7.0,788.166667,0.0,6.0,1129.196413,717.988423,0.0
2,10385163239,Sed Et Company,"([florists supplies, nursery stock, and flower...","florists supplies, nursery stock, and flowers",a,6.61,2.519346e+04,2.342538e+04,74.0,5.026389,...,1665.287911,1548.417412,0.067924,132.0,797.507171,0.0,73.0,345.115933,203.496326,3.0
3,10648956813,Proin Nisl Institute,"[(computeRs, computer peripheral equipment, an...","computers, computer peripheral equipment, and ...",a,6.66,1.419600e+06,1.329626e+06,21981.0,1390.109580,...,94545.379790,88553.112413,0.063241,2208.0,825.551750,1574.0,14380.0,98.720466,57.083817,710.0
4,10714068705,Sollicitudin Commodo Ipsum Industries,"([furniture, home furnishings and equipment sh...","furniture, home furnishings and equipment shop...",c,2.51,4.355100e+05,4.067113e+05,3438.0,221.458950,...,10931.299762,10208.454670,0.064415,1715.0,796.782831,10.0,3221.0,135.209547,113.737044,145.0


In [5]:
merchant_df['revenue_level'] = np.where(
    merchant_df['revenue_level'] == "a", 1,
    np.where(
        merchant_df['revenue_level'] == "b", 2,
        np.where(
            merchant_df['revenue_level'] == "c", 3,
            np.where(
                merchant_df['revenue_level'] == "d", 4,
                    5
            )
        )
    )
)

In [19]:
rank_cols_desc = [
    'revenue_level', 'take_rate', 'sales_revenue', 'discounted_sales_revenue', 
    'num_orders', 'avg_daily_rev', 'discounted_avg_daily_rev', 
    'avg_value_per_order', 'discounted_avg_value_per_order', 'avg_daily_orders',
    'avg_daily_commission', 'discounted_avg_daily_commission', 
    'avg_commission_per_order', 'discounted_avg_commission_per_order', 
    'overall_commission', 'discounted_overall_commission', 'sa2_region_count', 
    'median_customer_income', 'returning_customers', 'unique_customers', 
    'mean_spending', 'vip_customers'

]

rank_cols_asc = [
    'approximate_fraudulent_orders', 'avg_daily_approximate_fraudulent_orders', 
    'overall_fraud_rate', 'std_daily_revenue', 'std_daily_discounted_revenue',
    'std_spending'
]

# define the columns to add ranks for
rank_cols_dict = {}
for col in rank_cols_desc:
    rank_cols_dict[col] = False
for col in rank_cols_asc:
    rank_cols_dict[col] = True

# iterate through and rank these columns
for colname, asc in rank_cols_dict.items():
    merchant_df = RANK.add_column_rank(merchant_df, colname, ascending=asc,
        rank_type = 'minmax')

In [20]:
# calculate the uwar for the cols to rank
merchant_df = RANK.average_rank(merchant_df, rank_cols_dict.keys(),
    rank_type = 'minmax')

In [21]:
merchant_df = merchant_df.sort_values('average_rank').reset_index(drop = True)
merchant_df

,merchant_abn,name,tags,tag,revenue_level,take_rate,sales_revenue,discounted_sales_revenue,num_orders,approximate_fraudulent_orders,...,minmax_unique_customers,minmax_mean_spending,minmax_vip_customers,minmax_approximate_fraudulent_orders,minmax_avg_daily_approximate_fraudulent_orders,minmax_overall_fraud_rate,minmax_std_spending,average_rank,minmax_std_daily_revenue,minmax_std_daily_discounted_revenue
0,86578477987,Leo In Consulting,"[[watch, clock, and jewelry repair shops], [a]...","watch, clock, and jewelry repair shops",1,6.43,9.537907e+06,8.936672e+06,271130.0,16998.830670,...,0.000000,0.811342,0.229831,9.444129e-01,9.444129e-01,0.013599,0.104536,0.379564,0.684240,0.695206
1,32361057556,Orci In Consequat Corporation,"([gift, card, novelty, and souvenir shops], [a...","gift, card, novelty, and souvenir shops",1,6.61,9.434447e+06,8.823244e+06,85776.0,5497.318411,...,0.028447,0.807836,0.116323,3.054149e-01,3.054149e-01,0.045659,0.169279,0.386954,0.721681,0.732301
2,45629217853,Lacus Consulting,"[[gift, Card, novelty, and souvenir shops], [a...","gift, card, novelty, and souvenir shops",1,6.98,8.384278e+06,7.852651e+06,216673.0,13597.040991,...,0.000042,0.834743,0.131332,7.554172e-01,7.554172e-01,0.014923,0.115821,0.394743,0.643659,0.653795
3,21439773999,Mauris Non Institute,"([cable, satellite, and other pay television a...","cable, satellite, and other pay television and...",1,6.10,9.426503e+06,8.820947e+06,120497.0,7651.398835,...,0.006935,0.812267,0.126642,4.250907e-01,4.250907e-01,0.032067,0.154818,0.396521,0.737521,0.748516
4,48534649627,Dignissim Maecenas Foundation,"[[opticians, oPtical goods, and eyeglasses], [...","opticians, optical goods, and eyeglasses",1,6.64,9.406000e+06,8.772499e+06,65519.0,4239.362229,...,0.064701,0.800816,0.000000,2.355257e-01,2.355257e-01,0.059815,0.238013,0.397092,0.803961,0.813071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4021,34205660775,A Magna Associates,"[(antiQue shops - sales, repairs, and restorat...","antique shops - sales, repairs, and restoratio...",1,6.30,1.282631e+03,1.175776e+03,1.0,0.083310,...,1.000000,0.378154,1.000000,9.090969e-07,9.090969e-07,0.488012,NaN,NaN,NaN,NaN
4022,42461534060,Elit LLP,"([antique shops - sales, rePairs, and restOrat...","antique shops - sales, repairs, and restoratio...",2,3.98,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4023,52524795452,Cras Convallis Associates,"((antique shops - sales, repaIrs, and restorat...","antique shops - sales, repairs, and restoratio...",1,5.89,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4024,56738316313,Sollicitudin A LLP,"[(antique shops - sales, repairs, and restorat...","antique shops - sales, repairs, and restoratio...",3,1.97,1.942569e+03,1.753476e+03,1.0,0.097341,...,1.000000,0.055696,1.000000,1.688668e-06,1.688668e-06,0.810948,NaN,NaN,NaN,NaN


In [22]:
# choose the first 100 of these
final_100_df = merchant_df.head(100)
final_100_df

,merchant_abn,name,tags,tag,revenue_level,take_rate,sales_revenue,discounted_sales_revenue,num_orders,approximate_fraudulent_orders,...,minmax_unique_customers,minmax_mean_spending,minmax_vip_customers,minmax_approximate_fraudulent_orders,minmax_avg_daily_approximate_fraudulent_orders,minmax_overall_fraud_rate,minmax_std_spending,average_rank,minmax_std_daily_revenue,minmax_std_daily_discounted_revenue
0,86578477987,Leo In Consulting,"[[watch, clock, and jewelry repair shops], [a]...","watch, clock, and jewelry repair shops",1,6.43,9.537907e+06,8.936672e+06,271130.0,16998.830670,...,0.000000,0.811342,0.229831,0.944413,0.944413,0.013599,0.104536,0.379564,0.684240,0.695206
1,32361057556,Orci In Consequat Corporation,"([gift, card, novelty, and souvenir shops], [a...","gift, card, novelty, and souvenir shops",1,6.61,9.434447e+06,8.823244e+06,85776.0,5497.318411,...,0.028447,0.807836,0.116323,0.305415,0.305415,0.045659,0.169279,0.386954,0.721681,0.732301
2,45629217853,Lacus Consulting,"[[gift, Card, novelty, and souvenir shops], [a...","gift, card, novelty, and souvenir shops",1,6.98,8.384278e+06,7.852651e+06,216673.0,13597.040991,...,0.000042,0.834743,0.131332,0.755417,0.755417,0.014923,0.115821,0.394743,0.643659,0.653795
3,21439773999,Mauris Non Institute,"([cable, satellite, and other pay television a...","cable, satellite, and other pay television and...",1,6.10,9.426503e+06,8.820947e+06,120497.0,7651.398835,...,0.006935,0.812267,0.126642,0.425091,0.425091,0.032067,0.154818,0.396521,0.737521,0.748516
4,48534649627,Dignissim Maecenas Foundation,"[[opticians, oPtical goods, and eyeglasses], [...","opticians, optical goods, and eyeglasses",1,6.64,9.406000e+06,8.772499e+06,65519.0,4239.362229,...,0.064701,0.800816,0.000000,0.235526,0.235526,0.059815,0.238013,0.397092,0.803961,0.813071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,26169574842,Dolor Dolor Industries,"[[books, periodicals, and newspapers], [a], [t...","books, periodicals, and newspapers",1,6.58,3.206616e+06,2.978012e+06,9809.0,669.244785,...,0.664867,0.810743,0.616323,0.037178,0.037178,0.140902,0.217227,0.604347,0.379415,0.382322
96,83690644458,Id Erat Etiam Consulting,"[(gift, card, novelty, and souvenir shops), (b...","gift, card, novelty, and souvenir shops",2,3.15,3.183224e+06,2.979746e+06,35852.0,2283.113162,...,0.224917,0.921540,0.202627,0.126841,0.126841,0.036277,0.075110,0.605122,0.258292,0.262450
97,78760357380,Nibh Aliquam Corp.,"([computers, computer peripheral equipment, an...","computers, computer peripheral equipment, and ...",3,1.61,6.023001e+06,5.593174e+06,20212.0,1367.705767,...,0.433140,0.789287,0.381801,0.075983,0.075983,0.128023,0.266487,0.605189,0.611852,0.616388
98,46012371285,A Ultricies Inc.,"((books, periodicals, and newspapers), (b), (t...","books, periodicals, and newspapers",2,4.87,4.012275e+06,3.714571e+06,8218.0,585.597782,...,0.713164,0.721075,0.700750,0.032531,0.032531,0.210644,0.270430,0.606213,0.515350,0.517870


In [23]:
# save the rank, abn, name and sales_revenue in a csv
final_100_df[['merchant_abn', 'name']].to_csv('../data/top-100.csv')

In [24]:
import json
# read in the segments that Oliver defined
segments = json.load(open('../data/curated/segments.json'))
segments

{'leisure goods and services': [11944993446,
  13467303030,
  13823979247,
  14081759429,
  14148282104,
  14809247463,
  15253672771,
  15560455575,
  17507773571,
  17541067926,
  20422223827,
  20445424481,
  23541969839,
  24164521701,
  27326652377,
  28034382339,
  29917527785,
  32610248934,
  32897338221,
  37470904997,
  38700038932,
  38734016504,
  39759375662,
  40507108040,
  41333655501,
  42399949944,
  43698901053,
  43835684204,
  44461622420,
  45151159881,
  45899477665,
  46331355995,
  46877655150,
  47484801038,
  48294754935,
  48625977738,
  48854820141,
  49418428210,
  49460691968,
  49891706470,
  52827561456,
  53378405666,
  55179323922,
  57093429717,
  57798993346,
  58729395250,
  60616659469,
  60654402457,
  62170730910,
  63948574402,
  64185141673,
  66118124566,
  66257020651,
  66610548417,
  68514534266,
  68656010159,
  69655310165,
  70088042348,
  70172340121,
  71305424518,
  73004400052,
  73841664453,
  74753274268,
  75239540583,
  79306976

In [25]:
# create a mapping df real quick
rows = []

for seg, abn_list in segments.items():
    for abn in abn_list:
        rows.append({'segment': seg, 'merchant_abn': abn})

segments_df = pd.DataFrame(rows)
segments_df

,segment,merchant_abn
0,leisure goods and services,11944993446
1,leisure goods and services,13467303030
2,leisure goods and services,13823979247
3,leisure goods and services,14081759429
4,leisure goods and services,14148282104
...,...,...
3198,tech and telecom,91062920626
3199,tech and telecom,92594285117
3200,tech and telecom,93740493847
3201,tech and telecom,93932766844


In [26]:
# perform a join on top 100 abns and segments
segmented_100_df = final_100_df.join(
    segments_df.set_index('merchant_abn'),
    on = 'merchant_abn',
    rsuffix = '_1',
    how = 'left'
)
segmented_100_df

,merchant_abn,name,tags,tag,revenue_level,take_rate,sales_revenue,discounted_sales_revenue,num_orders,approximate_fraudulent_orders,...,minmax_mean_spending,minmax_vip_customers,minmax_approximate_fraudulent_orders,minmax_avg_daily_approximate_fraudulent_orders,minmax_overall_fraud_rate,minmax_std_spending,average_rank,minmax_std_daily_revenue,minmax_std_daily_discounted_revenue,segment
0,86578477987,Leo In Consulting,"[[watch, clock, and jewelry repair shops], [a]...","watch, clock, and jewelry repair shops",1,6.43,9.537907e+06,8.936672e+06,271130.0,16998.830670,...,0.811342,0.229831,0.944413,0.944413,0.013599,0.104536,0.379564,0.684240,0.695206,repair services
1,32361057556,Orci In Consequat Corporation,"([gift, card, novelty, and souvenir shops], [a...","gift, card, novelty, and souvenir shops",1,6.61,9.434447e+06,8.823244e+06,85776.0,5497.318411,...,0.807836,0.116323,0.305415,0.305415,0.045659,0.169279,0.386954,0.721681,0.732301,leisure goods and services
1,32361057556,Orci In Consequat Corporation,"([gift, card, novelty, and souvenir shops], [a...","gift, card, novelty, and souvenir shops",1,6.61,9.434447e+06,8.823244e+06,85776.0,5497.318411,...,0.807836,0.116323,0.305415,0.305415,0.045659,0.169279,0.386954,0.721681,0.732301,gifts souvenirs
2,45629217853,Lacus Consulting,"[[gift, Card, novelty, and souvenir shops], [a...","gift, card, novelty, and souvenir shops",1,6.98,8.384278e+06,7.852651e+06,216673.0,13597.040991,...,0.834743,0.131332,0.755417,0.755417,0.014923,0.115821,0.394743,0.643659,0.653795,leisure goods and services
2,45629217853,Lacus Consulting,"[[gift, Card, novelty, and souvenir shops], [a...","gift, card, novelty, and souvenir shops",1,6.98,8.384278e+06,7.852651e+06,216673.0,13597.040991,...,0.834743,0.131332,0.755417,0.755417,0.014923,0.115821,0.394743,0.643659,0.653795,gifts souvenirs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,83690644458,Id Erat Etiam Consulting,"[(gift, card, novelty, and souvenir shops), (b...","gift, card, novelty, and souvenir shops",2,3.15,3.183224e+06,2.979746e+06,35852.0,2283.113162,...,0.921540,0.202627,0.126841,0.126841,0.036277,0.075110,0.605122,0.258292,0.262450,leisure goods and services
96,83690644458,Id Erat Etiam Consulting,"[(gift, card, novelty, and souvenir shops), (b...","gift, card, novelty, and souvenir shops",2,3.15,3.183224e+06,2.979746e+06,35852.0,2283.113162,...,0.921540,0.202627,0.126841,0.126841,0.036277,0.075110,0.605122,0.258292,0.262450,gifts souvenirs
97,78760357380,Nibh Aliquam Corp.,"([computers, computer peripheral equipment, an...","computers, computer peripheral equipment, and ...",3,1.61,6.023001e+06,5.593174e+06,20212.0,1367.705767,...,0.789287,0.381801,0.075983,0.075983,0.128023,0.266487,0.605189,0.611852,0.616388,tech and telecom
98,46012371285,A Ultricies Inc.,"((books, periodicals, and newspapers), (b), (t...","books, periodicals, and newspapers",2,4.87,4.012275e+06,3.714571e+06,8218.0,585.597782,...,0.721075,0.700750,0.032531,0.032531,0.210644,0.270430,0.606213,0.515350,0.517870,leisure goods and services


In [27]:
segmented_100_df.groupby('segment').count()

,merchant_abn,name,tags,tag,revenue_level,take_rate,sales_revenue,discounted_sales_revenue,num_orders,approximate_fraudulent_orders,...,minmax_unique_customers,minmax_mean_spending,minmax_vip_customers,minmax_approximate_fraudulent_orders,minmax_avg_daily_approximate_fraudulent_orders,minmax_overall_fraud_rate,minmax_std_spending,average_rank,minmax_std_daily_revenue,minmax_std_daily_discounted_revenue
segment,,,,,,,,,,,,,,,,,,,,,
gifts souvenirs,13,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
home furnishings,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
leisure goods and services,45,45,45,45,45,45,45,45,45,45,...,45,45,45,45,45,45,45,45,45,45
repair services,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
tech and telecom,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15


In [28]:
segmented_100_df[segmented_100_df['segment'] == 'repair services']

,merchant_abn,name,tags,tag,revenue_level,take_rate,sales_revenue,discounted_sales_revenue,num_orders,approximate_fraudulent_orders,...,minmax_mean_spending,minmax_vip_customers,minmax_approximate_fraudulent_orders,minmax_avg_daily_approximate_fraudulent_orders,minmax_overall_fraud_rate,minmax_std_spending,average_rank,minmax_std_daily_revenue,minmax_std_daily_discounted_revenue,segment
0,86578477987,Leo In Consulting,"[[watch, clock, and jewelry repair shops], [a]...","watch, clock, and jewelry repair shops",1,6.43,9.537907e+06,8.936672e+06,271130.0,16998.830670,...,0.811342,0.229831,0.944413,0.944413,0.013599,0.104536,0.379564,0.684240,0.695206,repair services
12,49322182190,Gravida Mauris Incorporated,"([watch, clock, and jewelry repair shops], [a]...","watch, clock, and jewelry repair shops",1,6.35,7.854984e+06,7.343056e+06,51903.0,3368.324666,...,0.825152,0.155722,0.187133,0.187133,0.064238,0.152569,0.448347,0.642002,0.651316,repair services
30,68559320474,Aliquam Auctor Associates,"([antique shops - sales, repairs, and restorat...","antique shops - sales, repairs, and restoratio...",2,4.20,7.672531e+06,7.155022e+06,35551.0,2349.097643,...,0.802631,0.172608,0.130507,0.130507,0.091403,0.205471,0.506021,0.655287,0.663060,repair services
60,23338656015,Iaculis LLC,"([watch, clock, And jewelry repair shops], [b]...","watch, clock, and jewelry repair shops",2,3.25,5.746652e+06,5.368024e+06,39695.0,2572.530740,...,0.860654,0.151032,0.142920,0.142920,0.062187,0.151231,0.557944,0.486228,0.492904,repair services
74,62224020443,Hendrerit A Corporation,"((watch, clock, and jewelry repair shops), (a)...","watch, clock, and jewelry repair shops",1,6.64,3.399175e+06,3.173572e+06,19553.0,1276.830258,...,0.880943,0.383677,0.070934,0.070934,0.073547,0.125063,0.581861,0.319251,0.323606,repair services
75,29616684420,Tellus Id LLC,"((watch, clock, and jewelry repair shops), (b...","watch, clock, and jewelry repair shops",2,3.50,3.851749e+06,3.602111e+06,44316.0,2823.604444,...,0.911674,0.085366,0.156869,0.156869,0.037051,0.107032,0.582284,0.337866,0.342894,repair services
76,81761494572,Nulla Facilisis Institute,"([watch, clock, and jewelry repaIr shops], [a]...","watch, clock, and jewelry repair shops",1,6.82,3.031698e+06,2.833605e+06,26249.0,1685.364235,...,0.912236,0.307692,0.093631,0.093631,0.048364,0.103932,0.582435,0.270190,0.274178,repair services
92,71528203369,Ipsum Primis Associates,"((watch, clock, and jewelry repair shops), (a)...","watch, clock, and jewelry repair shops",1,6.94,1.878869e+06,1.758205e+06,31370.0,1981.043827,...,0.952757,0.163227,0.110059,0.110059,0.024063,0.069845,0.602835,0.172198,0.174914,repair services
93,76626119831,Tristique Pellentesque Inc.,"[[antique shops - sales, repairs, and restorat...","antique shops - sales, repairs, and restoratio...",3,1.99,3.840609e+06,3.589443e+06,35756.0,2289.692651,...,0.904245,0.146341,0.127206,0.127206,0.044447,0.120429,0.603396,0.344270,0.349098,repair services


In [29]:
# perform a join on merchant abns and segments
segmented_merchant_df = merchant_df.join(
    segments_df.set_index('merchant_abn'),
    on = 'merchant_abn',
    rsuffix = '_1',
    how = 'left'
)
segmented_merchant_df

,merchant_abn,name,tags,tag,revenue_level,take_rate,sales_revenue,discounted_sales_revenue,num_orders,approximate_fraudulent_orders,...,minmax_mean_spending,minmax_vip_customers,minmax_approximate_fraudulent_orders,minmax_avg_daily_approximate_fraudulent_orders,minmax_overall_fraud_rate,minmax_std_spending,average_rank,minmax_std_daily_revenue,minmax_std_daily_discounted_revenue,segment
0,86578477987,Leo In Consulting,"[[watch, clock, and jewelry repair shops], [a]...","watch, clock, and jewelry repair shops",1,6.43,9.537907e+06,8.936672e+06,271130.0,16998.830670,...,0.811342,0.229831,9.444129e-01,9.444129e-01,0.013599,0.104536,0.379564,0.684240,0.695206,repair services
1,32361057556,Orci In Consequat Corporation,"([gift, card, novelty, and souvenir shops], [a...","gift, card, novelty, and souvenir shops",1,6.61,9.434447e+06,8.823244e+06,85776.0,5497.318411,...,0.807836,0.116323,3.054149e-01,3.054149e-01,0.045659,0.169279,0.386954,0.721681,0.732301,leisure goods and services
1,32361057556,Orci In Consequat Corporation,"([gift, card, novelty, and souvenir shops], [a...","gift, card, novelty, and souvenir shops",1,6.61,9.434447e+06,8.823244e+06,85776.0,5497.318411,...,0.807836,0.116323,3.054149e-01,3.054149e-01,0.045659,0.169279,0.386954,0.721681,0.732301,gifts souvenirs
2,45629217853,Lacus Consulting,"[[gift, Card, novelty, and souvenir shops], [a...","gift, card, novelty, and souvenir shops",1,6.98,8.384278e+06,7.852651e+06,216673.0,13597.040991,...,0.834743,0.131332,7.554172e-01,7.554172e-01,0.014923,0.115821,0.394743,0.643659,0.653795,leisure goods and services
2,45629217853,Lacus Consulting,"[[gift, Card, novelty, and souvenir shops], [a...","gift, card, novelty, and souvenir shops",1,6.98,8.384278e+06,7.852651e+06,216673.0,13597.040991,...,0.834743,0.131332,7.554172e-01,7.554172e-01,0.014923,0.115821,0.394743,0.643659,0.653795,gifts souvenirs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4021,34205660775,A Magna Associates,"[(antiQue shops - sales, repairs, and restorat...","antique shops - sales, repairs, and restoratio...",1,6.30,1.282631e+03,1.175776e+03,1.0,0.083310,...,0.378154,1.000000,9.090969e-07,9.090969e-07,0.488012,NaN,NaN,NaN,NaN,repair services
4022,42461534060,Elit LLP,"([antique shops - sales, rePairs, and restOrat...","antique shops - sales, repairs, and restoratio...",2,3.98,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,repair services
4023,52524795452,Cras Convallis Associates,"((antique shops - sales, repaIrs, and restorat...","antique shops - sales, repairs, and restoratio...",1,5.89,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,repair services
4024,56738316313,Sollicitudin A LLP,"[(antique shops - sales, repairs, and restorat...","antique shops - sales, repairs, and restoratio...",3,1.97,1.942569e+03,1.753476e+03,1.0,0.097341,...,0.055696,1.000000,1.688668e-06,1.688668e-06,0.810948,NaN,NaN,NaN,NaN,repair services


In [30]:
segmented_merchant_df.groupby('segment').count()

,merchant_abn,name,tags,tag,revenue_level,take_rate,sales_revenue,discounted_sales_revenue,num_orders,approximate_fraudulent_orders,...,minmax_unique_customers,minmax_mean_spending,minmax_vip_customers,minmax_approximate_fraudulent_orders,minmax_avg_daily_approximate_fraudulent_orders,minmax_overall_fraud_rate,minmax_std_spending,average_rank,minmax_std_daily_revenue,minmax_std_daily_discounted_revenue
segment,,,,,,,,,,,,,,,,,,,,,
gifts souvenirs,362,362,362,362,362,362,362,362,362,362,...,362,362,362,362,362,362,362,362,362,362
home furnishings,649,649,649,649,649,649,648,648,648,648,...,648,648,648,648,648,648,642,642,642,642
leisure goods and services,1221,1221,1221,1221,1221,1221,1221,1221,1221,1221,...,1221,1221,1221,1221,1221,1221,1221,1221,1221,1221
repair services,299,299,299,299,299,299,249,249,249,249,...,249,249,249,249,249,249,240,240,240,240
tech and telecom,672,672,672,672,672,672,666,666,666,666,...,666,666,666,666,666,666,661,661,661,661


In [31]:
for segment in set(segmented_merchant_df['segment']):
    print(f'\nTop 10 {segment} Merchants')
    print(segmented_merchant_df[ segmented_merchant_df['segment'] == segment].head(10)[['merchant_abn', 'name']])


Top 10 nan Merchants
Empty DataFrame
Columns: [merchant_abn, name]
Index: []

Top 10 gifts souvenirs Merchants
    merchant_abn                            name
1    32361057556   Orci In Consequat Corporation
2    45629217853                Lacus Consulting
7    94493496784   Dictum Phasellus In Institute
8    79417999332            Phasellus At Company
10   43186523025  Lorem Ipsum Sodales Industries
16   60956456424         Ultricies Dignissim LLP
17   24852446429                  Erat Vitae LLP
21   49212265466                  Auctor Company
43   21772962346      Purus Gravida Sagittis Ltd
46   80324045558     Ipsum Dolor Sit Corporation

Top 10 leisure goods and services Merchants
    merchant_abn                           name
1    32361057556  Orci In Consequat Corporation
2    45629217853               Lacus Consulting
5    64403598239      Lobortis Ultrices Company
6    89726005175            Est Nunc Consulting
7    94493496784  Dictum Phasellus In Institute
8    79417999332

## CURRENT TOP 10s (to be tweaked)


### Top 10 tech and telecom Merchants
```
    merchant_abn                             name
8    68216911708  Placerat Eget Venenatis Limited
10   21439773999             Mauris Non Institute
21   35909341340         Arcu Sed Eu Incorporated
38   80518954462    Neque Sed Dictum Incorporated
45   34096466752                  Nullam Enim Ltd
51   84703983173                  Amet Consulting
52   94690988633                  Eu Placerat LLC
63   45433476494       Adipiscing Elit Foundation
64   67400260923                      Eleifend PC
66   77590625261              Sed Diam Foundation
```

### Top 10 gifts souvenirs Merchants
```
    merchant_abn                            name
1    24852446429                  Erat Vitae LLP
3    45629217853                Lacus Consulting
6    43186523025  Lorem Ipsum Sodales Industries
9    80324045558     Ipsum Dolor Sit Corporation
12   79417999332            Phasellus At Company
14   32361057556   Orci In Consequat Corporation
15   94493496784   Dictum Phasellus In Institute
18   60956456424         Ultricies Dignissim LLP
33   49212265466                  Auctor Company
61   81219314324                Faucibus Leo Ltd
```

### Top 10 leisure goods and services Merchants
```
    merchant_abn                               name
2    64203420245                 Pede Nonummy Corp.
3    45629217853                   Lacus Consulting
4    89726005175                Est Nunc Consulting
7    49891706470          Non Vestibulum Industries
9    80324045558        Ipsum Dolor Sit Corporation
11   64403598239          Lobortis Ultrices Company
12   79417999332               Phasellus At Company
13   63290521567  Vehicula Pellentesque Corporation
14   32361057556      Orci In Consequat Corporation
15   94493496784      Dictum Phasellus In Institute
```
### Top 10 home furnishings Merchants
```
    merchant_abn                            name
1    24852446429                  Erat Vitae LLP
6    43186523025  Lorem Ipsum Sodales Industries
20   76767266140            Phasellus At Limited
27   38090089066       Interdum Feugiat Sed Inc.
32   90543168331  Phasellus Dapibus Incorporated
33   49212265466                  Auctor Company
39   52959528548               Libero Et Limited
40   50315283629        Iaculis Aliquet Diam LLC
62   79827781481                 Amet Risus Inc.
69   98269572896         Eget Laoreet Posuere PC
```

### Top 10 repair services Merchants
```
     merchant_abn                         name
0     86578477987            Leo In Consulting
28    49322182190  Gravida Mauris Incorporated
41    68559320474    Aliquam Auctor Associates
65    23338656015                  Iaculis LLC
74    29616684420                Tellus Id LLC
77    76626119831  Tristique Pellentesque Inc.
104   30280912400            Mauris Erat Corp.
114   34920839316                 Nunc Est LLP
128   49063322245   Fringilla Donec Associates
142   80779820715             Euismod Enim LLC
```